In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Recommended_System").getOrCreate()

In [19]:

df = spark.read.option("header", True) \
               .option("inferSchema", True) \
               .option("multiLine", True) \
               .option("escape", '"') \
               .csv("eCommerce-dataset-samples\walmart-products.csv")  # Đổi thành đường dẫn file của bạn

df.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "d:\anaconda3\envs\DE\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\envs\DE\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda3\envs\DE\Lib\socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df = spark.read.format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("multiLine", True) \
    .load("eCommerce-dataset-samples\walmart-products.csv")
df.show()

+--------------------+--------------------+--------------------+----------+--------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+------------------+--------------------+--------------------+------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+------+----------+-----+-----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+--------------------+
|           timestamp|                 url|         final_price|       sku|currency|        gtin|      specifications|          image_urls|         top_reviews|        rating_stars|       related_pages|ava

In [ ]:
df.select("specifications").show(5)

+--------------------+
|      specifications|
+--------------------+
|[{"name":"Brand",...|
|[{"name":"Brand",...|
|[{"name":"Feature...|
|[{"name":"Brand",...|
|[{"name":"Country...|
+--------------------+
only showing top 5 rows



In [ ]:
df.select("specifications").show(5)

+--------------------+
|      specifications|
+--------------------+
|[{"name":"Brand",...|
|[{"name":"Brand",...|
|[{"name":"Feature...|
|[{"name":"Brand",...|
|[{"name":"Country...|
+--------------------+
only showing top 5 rows



In [ ]:
df.select("top_reviews").collect()

In [ ]:
df.select("rating_stars").collect()

In [ ]:
set(df.select("categories").collect())

In [ ]:
df.describe().show()

+-------+--------------------+-----------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+----+-----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------+--------------------+--------------------+
|summary|                 url|      final_price|                 sku|currency|                gtin|      specifications|          image_urls|         top_reviews|        rating_stars|       related_pages|  

In [ ]:
df.select('rating_stars').show()

+--------------------+
|        rating_stars|
+--------------------+
|{"five_stars":2,"...|
|{"five_stars":47,...|
|    {"five_stars":2}|
|{"five_stars":78,...|
|{"five_stars":12,...|
|{"five_stars":63,...|
|{"five_stars":8,"...|
|{"five_stars":228...|
|{"five_stars":164...|
|{"five_stars":58,...|
|{"five_stars":534...|
|{"five_stars":42,...|
|{"five_stars":177...|
|    {"five_stars":8}|
|{"five_stars":7,"...|
|    {"five_stars":9}|
|{"five_stars":1,"...|
|{"five_stars":50,...|
|{"five_stars":424...|
|{"five_stars":71,...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col, expr, from_json
from pyspark.sql.types import MapType, StringType, IntegerType# Định nghĩa kiểu dữ liệu cho JSON
schema = MapType(StringType(), IntegerType())

# Chuyển đổi chuỗi JSON thành MapType
df = df.withColumn("rating_map", from_json(col("rating_stars"), schema))

# Tính tổng điểm và tổng số đánh giá
df = df.withColumn("total_points",
                   expr("COALESCE(rating_map['five_stars'],0) * 5 + "
                        "COALESCE(rating_map['four_stars'],0) * 4 + "
                        "COALESCE(rating_map['three_stars'],0) * 3 + "
                        "COALESCE(rating_map['two_stars'],0) * 2 + "
                        "COALESCE(rating_map['one_star'],0) * 1"))

df = df.withColumn("total_reviews",
                   expr("COALESCE(rating_map['five_stars'],0) + "
                        "COALESCE(rating_map['four_stars'],0) + "
                        "COALESCE(rating_map['three_stars'],0) + "
                        "COALESCE(rating_map['two_stars'],0) + "
                        "COALESCE(rating_map['one_star'],0)"))

# Tính trung bình rating
df = df.withColumn("average_rating", col("total_points") / col("total_reviews"))

# Hiển thị kết quả
df.select("rating_stars", "average_rating").show(truncate=False)

+-------------------------------------------------------------------------------------+-----------------+
|rating_stars                                                                         |average_rating   |
+-------------------------------------------------------------------------------------+-----------------+
|{"five_stars":2,"four_stars":3,"two_stars":1}                                        |4.0              |
|{"five_stars":47,"four_stars":4,"one_star":4,"three_stars":3}                        |4.551724137931035|
|{"five_stars":2}                                                                     |5.0              |
|{"five_stars":78,"four_stars":17,"one_star":2,"three_stars":3,"two_stars":1}         |4.663366336633663|
|{"five_stars":12,"one_star":1,"three_stars":1,"two_stars":1}                         |4.4              |
|{"five_stars":63,"four_stars":21,"one_star":5,"three_stars":5,"two_stars":1}         |4.431578947368421|
|{"five_stars":8,"four_stars":2,"one_star":1,"

In [ ]:
from pyspark.sql.functions import col, regexp_replace, floor

# Loại bỏ dấu $ và chuyển về kiểu số
df = df.withColumn("discount", regexp_replace(col("discount"), "[$]", "").cast("double"))
df = df.withColumn("initial_price", regexp_replace(col("initial_price"), "[$]", "").cast("double"))

# Tính phần trăm giảm giá và làm tròn xuống số nguyên
df = df.withColumn("percent_discount", floor((col("discount") / col("initial_price")) * 100))

In [ ]:
df.show()

+-------------------+--------------------+-----------+----------+--------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+------------------+--------------------+--------------------+------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+------+----------+-----+-----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+--------------------+--------------------+------------+-------------+-----------------+----------------+
|          timestamp|                 url|final_price|       sku|currency|        gtin|      specifications|          image_urls|   

Item sort by percentage \
Max discount percentage is 83% \
Item with no discount, default is 0%

In [ ]:
df = df.fillna({'percent_discount': 0})  # Thay NaN bằng 0
sorted_df = df.orderBy(df['percent_discount'].desc())
sorted_df.select('percent_discount').describe().show()


+-------+------------------+
|summary|  percent_discount|
+-------+------------------+
|  count|              1000|
|   mean|            25.118|
| stddev|18.614364629784447|
|    min|                 0|
|    max|                83|
+-------+------------------+



In [ ]:
df.select('final_price').describe().show()

+-------+-----------------+
|summary|      final_price|
+-------+-----------------+
|  count|             1000|
|   mean|37.98634000000022|
| stddev| 97.4809396210038|
|    min|             0.91|
|    max|          1685.99|
+-------+-----------------+



In [ ]:
df.select('categories').show()

+--------------------+
|          categories|
+--------------------+
|["Beauty","Makeup...|
|["Home","Decor","...|
|["Clothing","Wome...|
|["Home","Bedding"...|
|["Clothing","Kids...|
|["Clothing","Wome...|
|["Clothing","Wome...|
|["Clothing","Wome...|
|["Clothing","Wome...|
|["Clothing","Mens...|
|["Beauty","Makeup...|
|["Home","Decor","...|
|["Clothing","Shoe...|
|["Home","Bedding"...|
|["Clothing","Wome...|
|["Clothing","Grap...|
|["Clothing","Wome...|
|["Clothing","Wome...|
|["Home","Bedding"...|
|["Sports & Outdoo...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import ArrayType, StringType

def count_products_by_main_category(df):
    # Định nghĩa schema cho cột JSON
    schema = ArrayType(StringType())
    
    # Chuyển đổi chuỗi JSON thành danh sách thực tế
    df = df.withColumn("categories", from_json(col("categories"), schema))
    
    # Lấy category đầu tiên (category cha)
    df = df.withColumn("main_category", col("categories")[0])
    
    # Nhóm theo category cha và đếm số lượng sản phẩm
    category_counts = df.groupBy("main_category").count()
    
    return category_counts

new_df = count_products_by_main_category(df)
new_df.show()


+--------------------+-----+
|       main_category|count|
+--------------------+-----+
|    Home Improvement|    3|
|                Home|  220|
|                Food|   29|
|       Subscriptions|    1|
|      Premium Beauty|    5|
|            Seasonal|    1|
|                Baby|   13|
|        Auto & Tires|    1|
|   Sports & Outdoors|   12|
|Household Essentials|    3|
|         Electronics|    1|
|   Shop with Purpose|    1|
|            Clothing|  594|
|      Patio & Garden|   14|
|                Pets|   17|
| Health and Medicine|   14|
|        Collectibles|    1|
|       Personal Care|   16|
|             Jewelry|    9|
|              Beauty|   31|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col, from_json, size, max

def max_category_count(df):
    # Định nghĩa schema cho cột JSON
    schema = ArrayType(StringType())
    
    # Chuyển đổi chuỗi JSON thành danh sách thực tế
    df = df.withColumn("categories", from_json(col("categories"), schema))
    
    # Tính số lượng category của mỗi item
    df = df.withColumn("category_count", size(col("categories")))
    
    # Lấy giá trị lớn nhất
    max_count = df.agg(max("category_count")).collect()[0][0]
    
    return max_count

max_count  = max_category_count(df)
max_count

7

In [ ]:
df = df.fillna({'percent_discount': 0})  # Thay NaN bằng 0
sorted_df = df.orderBy(df['percent_discount'].desc())
sorted_df.select('percent_discount').describe().show()


+-------+------------------+
|summary|  percent_discount|
+-------+------------------+
|  count|              1000|
|   mean|            25.118|
| stddev|18.614364629784447|
|    min|                 0|
|    max|                83|
+-------+------------------+



In [ ]:
df = df.fillna({'percent_discount': 0})  # Thay NaN bằng 0
sorted_df = df.orderBy(df['percent_discount'].desc())
sorted_df.select('percent_discount').describe().show()


+-------+------------------+
|summary|  percent_discount|
+-------+------------------+
|  count|              1000|
|   mean|            25.118|
| stddev|18.614364629784447|
|    min|                 0|
|    max|                83|
+-------+------------------+



In [ ]:
df = df.fillna({'percent_discount': 0})  # Thay NaN bằng 0
sorted_df = df.orderBy(df['percent_discount'].desc())
sorted_df.select('percent_discount').describe().show()


+-------+------------------+
|summary|  percent_discount|
+-------+------------------+
|  count|              1000|
|   mean|            25.118|
| stddev|18.614364629784447|
|    min|                 0|
|    max|                83|
+-------+------------------+



In [ ]:
count_df = df.groupBy("available_for_delivery").count()
count_df.show()

+----------------------+-----+
|available_for_delivery|count|
+----------------------+-----+
|                  true|  977|
|                 false|   23|
+----------------------+-----+



In [ ]:
count_df = df.groupBy("available_for_pickup").count()
count_df.show()

+--------------------+-----+
|available_for_pickup|count|
+--------------------+-----+
|                true|   48|
|               false|  952|
+--------------------+-----+



In [ ]:
brand_count = df.select("brand").distinct().count()
brand_count 

505

we can see that there is 505 brands in total 1000 item, so that the brand is not a good feature for the model 

In [ ]:
specification_df = df.select("specifications")
specification_df.show()

+--------------------+
|      specifications|
+--------------------+
|[{"name":"Brand",...|
|[{"name":"Brand",...|
|[{"name":"Feature...|
|[{"name":"Brand",...|
|[{"name":"Country...|
|[{"name":"Clothin...|
|[{"name":"Feature...|
|[{"name":"Feature...|
|[{"name":"Feature...|
|[{"name":"Brand",...|
|[{"name":"Brand",...|
|[{"name":"Country...|
|[{"name":"Brand",...|
|[{"name":"Brand",...|
|[{"name":"Feature...|
|[{"name":"Country...|
|[{"name":"Feature...|
|[{"name":"Country...|
|[{"name":"Brand",...|
|[{"name":"Country...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col, from_json, explode
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# Định nghĩa schema cho specifications (mảng các struct chứa name và value)
schema = ArrayType(StructType([
    StructField("name", StringType(), True),
    StructField("value", StringType(), True)
]))

# Chuyển cột specifications từ STRING thành JSON ARRAY
df_parsed = df.withColumn("specifications", from_json(col("specifications"), schema))

# Giải nén danh sách specifications và chỉ lấy "name"
df_keys = df_parsed.withColumn("name", explode(col("specifications.name"))).select("name").distinct()

# Chuyển về danh sách Python
key_set = set(df_keys.toPandas()["name"].tolist())

# In ra kết quả
key_set

{'Absorbency',
 'Age Group',
 'Age Range',
 'Allergen-Free Statements',
 'Animal Breed',
 'Animal Lifestage',
 'Animal Type',
 'Animal Welfare Claims',
 'Assembled Product Dimensions (L x W x H)',
 'Assembled Product Weight',
 'Assembly Instructions',
 'Baby Food Packaging',
 'Baby Food Stage',
 'Backing Material',
 'Battery Life',
 'Bed Frame Type',
 'Bed Headboard & Footboard Type',
 'Bed Pillow Size',
 'Bed Sheet Type',
 'Bed Size',
 'Bed Type',
 'Bedding Set Type',
 'Body Part',
 'Body Wash & Shower Gel Type',
 'Boot Shaft Width Style',
 'Boot Style',
 'Brand',
 'Burn Time',
 'Caffeine Designation',
 'Candle Type',
 'Candle Wax Type',
 'Carats',
 'Casual & Dress Shoe Style',
 'Character',
 'Cleaning, Care & Maintenance',
 'Clothing Fit',
 'Clothing Length Style',
 'Clothing Neck Style',
 'Clothing Size',
 'Clothing Size Group',
 'Clothing Style',
 'Coffee Bean Origin',
 'Collection',
 'Color',
 'Color Category',
 'Condition',
 'Container Material',
 'Container Type',
 'Count',
 'Co

In [ ]:
from pyspark.sql.functions import col, split, explode

# Tách danh sách JSON thành mảng
extracted_df = df.withColumn("categories", split(col("categories"), ",").getItem(0))

# Loại bỏ dấu ngoặc và khoảng trắng dư thừa
cleaned_df = extracted_df.withColumn("categories", col("categories").substr(2, 100))

# Đếm số loại khác nhau
category_count = cleaned_df.select("categories").distinct().count()

# Hiển thị danh sách loại khác nhau
cleaned_df.select("categories").distinct().show(truncate=False)
print(f"Tổng số loại khác nhau: {category_count}")


+----------------------+
|categories            |
+----------------------+
|"Food"                |
|"Arts Crafts & Sewing"|
|"Beauty"              |
|"Party & Occasions"   |
|"Home Improvement"    |
|"Shop with Purpose"   |
|"Seasonal"            |
|"Subscriptions"       |
|"Jewelry"             |
|"Auto & Tires"        |
|"Home"                |
|"Toys"                |
|"Collectibles"        |
|"Premium Beauty"      |
|"Patio & Garden"      |
|"Sports & Outdoors"   |
|"Household Essentials"|
|"Baby"                |
|"Electronics"         |
|"Health and Medicine" |
+----------------------+
only showing top 20 rows

Tổng số loại khác nhau: 23


In [ ]:
df.orderBy("categories").select("categories").show()

+--------------------+
|          categories|
+--------------------+
|["Arts Crafts & S...|
|["Arts Crafts & S...|
|["Auto & Tires","...|
|["Baby","Diaperin...|
|["Baby","Feeding"...|
|["Baby","Health &...|
|["Baby","Health &...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Baby","Toddlers...|
|["Beauty","Beauty...|
|["Beauty","Beauty...|
|["Beauty","Beauty...|
|["Beauty","Beauty...|
+--------------------+
only showing top 20 rows



In [ ]:
from collections import OrderedDict

categories_list = sorted(set(row["categories"] for row in df.select("categories").distinct().collect()))
categories_ordered_set = OrderedDict.fromkeys(categories_list)

categories_ordered_set # Output giữ thứ tự alphabet


OrderedDict([('["Arts Crafts & Sewing","Fabric","Shop All Fabric"]', None),
             ('["Arts Crafts & Sewing","Scrapbooking","Stickers"]', None),
             ('["Auto & Tires","Interior Parts & Accessories","Car Seat Cushions","Car Seat Cushions by Material","Air Car Seat Cushions"]',
              None),
             ('["Baby","Diapering","Diapers","Size 4 Diapers"]', None),
             ('["Baby","Feeding","Baby Food","Baby Snack Foods"]', None),
             ('["Baby","Health & Safety","Baby Gates","Retractable baby gates"]',
              None),
             ('["Baby","Health & Safety","Baby Medicine Cabinet","Baby Itching & Rash Treatments"]',
              None),
             ('["Baby","Toddlers\' Room","Toddler Bedding","Toddler Bedding Sets & Sheets"]',
              None),
             ('["Beauty","Beauty by Top Brands","Aveeno","Aveeno Hair Care"]',
              None),
             ('["Beauty","Beauty by Top Brands","Clean & Clear"]', None),
             ('["Beauty","B

In [ ]:
categories_sorted_set = tuple(sorted(set(row["categories"] for row in df.select("categories").distinct().collect())))
count = len(categories_sorted_set)

print(count)  # Số lượng phần tử trong set

452


In [ ]:
#lấy cac cột sau product_name, final_price, available_for_delivery, available_for_pickup, review_count, brand , review_tags, categories, rating, size, percentage_discount, original_price, discount, discount_amount, categories

In [ ]:
df.show()

+-------------------+--------------------+-----------+----------+--------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+------------------+--------------------+--------------------+------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+------+----------+-----+-----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+--------------------+
|          timestamp|                 url|final_price|       sku|currency|        gtin|      specifications|          image_urls|         top_reviews|        rating_stars|       related_pages|available_for_delivery|

In [ ]:
Collaborative Filtering
Decision Tree 
Random Forest( Decision Tree cho từng chunk data)

In [ ]:
from pyspark.sql.functions import col

# Xác định số lượng cấp tối đa
max_depth = df.selectExpr("size(categories) as depth").agg({"depth": "max"}).collect()[0][0]
# Tạo các cột sub_category_x tương ứng với từng cấp danh mục
for i in range(max_depth):
    df = df.withColumn(f"sub_category_{i}", col("categories")[i])

# Xóa cột categories gốc
df = df.drop("categories")

df.show(truncate=False)



+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+--------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df.show()

+-------------------+--------------------+-----------+----------+--------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+------------------+--------------------+--------------------+------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+------+----------+-----+-----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-----------------+--------------+--------------+--------------+
|          timestamp|                 url|final_price|       sku|curren

In [ ]:
import json
import pandas as pd

# Đọc dữ liệu từ file CSV (giả sử có cột 'review_tag')
df = pd.read_csv("eCommerce-dataset-samples\walmart-products.csv")  # Thay bằng đường dẫn file thực tế

# Hàm để trích xuất tags từ một list JSON
def extract_tags(json_list):
    try:
        tag_list = json.loads(json_list)  # Chuyển từ chuỗi JSON sang danh sách
        return {tag.split(" (")[0] for tag in tag_list}  # Lấy phần tag (bỏ số trong ngoặc)
    except:
        return set()

# Áp dụng hàm extract_tags cho cột review_tag
all_tags = set()
df["review_tags"].dropna().apply(lambda x: all_tags.update(extract_tags(x)))

# In ra danh sách tất cả các tag duy nhất
all_tags = sorted(all_tags)
all_tags
# print(len(all_tags))


["'s Finish",
 'Absorbency',
 'Absorbent',
 'Absorbing',
 'Accessories',
 'Aches',
 'Acne',
 'Active Wear',
 'Additives',
 'Adhesive',
 'Adjustable Height',
 'Adjustment',
 'Adults',
 'Advertisement',
 'Advocate',
 'Aesthetics',
 'Almonds',
 'Ankle',
 'Ankles',
 'Appearance',
 'Applesauce',
 'Application',
 'Applicator',
 'Applying',
 'Arch',
 'Armholes',
 'Arms',
 'As A Gift',
 'Assembly',
 'Assortment',
 'Atmosphere',
 'Attachments',
 'Babies',
 'Baby',
 'Back',
 'Backing',
 'Backpack',
 'Backrest',
 'Backs',
 'Backyard',
 'Bacteria',
 'Bag',
 'Baking',
 'Balance',
 'Balcony',
 'Bali',
 'Band',
 'Bare Feet',
 'Barrier',
 'Bars',
 'Base',
 'Basket',
 'Bath',
 'Bathroom',
 'Battery',
 'Beach',
 'Bed',
 'Bedroom',
 'Beef',
 'Belly',
 'Belongings',
 'Belt',
 'Benefits',
 'Beverage',
 'Big Guy',
 'Bite',
 'Black',
 'Black Out',
 'Blackout',
 'Bladder',
 'Blanket',
 'Blazer',
 'Bleeding',
 'Bleeding Gums',
 'Blend',
 'Block',
 'Blonde',
 'Blue',
 'Blueberries',
 'Body',
 'Bold',
 'Bones',


In [ ]:
import json
import pandas as pd
from collections import Counter

# Giả sử bạn có một DataFrame với cột 'review_tag'
df = pd.read_csv("eCommerce-dataset-samples\walmart-products.csv")  # Thay bằng đường dẫn file thực tế

# Hàm để trích xuất tags từ một list JSON
def extract_tags(json_list):
    try:
        tag_list = json.loads(json_list)  # Chuyển đổi từ chuỗi JSON sang danh sách
        return [tag.split(" (")[0] for tag in tag_list]  # Lấy phần tag (bỏ số trong ngoặc)
    except:
        return []

# Áp dụng hàm extract_tags cho cột review_tag
df["extracted_tags"] = df["review_tags"].apply(extract_tags)

# Gộp tất cả tags vào một danh sách
all_tags = sum(df["extracted_tags"].tolist(), [])

# Đếm số lần xuất hiện của mỗi tag
tag_counts = Counter(all_tags)

# Chuyển thành DataFrame để xem kết quả
df_tags = pd.DataFrame(tag_counts.items(), columns=["Tag", "Count"])
df_tags = df_tags.sort_values(by="Count", ascending=False)

print(df_tags)

             Tag  Count
29           Fit    429
9        Quality    407
43          Size    407
35       Comfort    381
11    Appearance    372
...          ...    ...
1043    Capacity      1
1044  Belongings      1
1046        Wall      1
1047     Matting      1
1048     Scratch      1

[1060 rows x 2 columns]


In [ ]:
#Gom các tag thành các nhóm chung 
# !pip install spacy 
# !python -m spacy download en_core_web_md

import spacy

# Load mô hình ngôn ngữ (có thể dùng 'en_core_web_md' hoặc 'en_core_web_lg' để có vector tốt hơn)
nlp = spacy.load("en_core_web_md")
tags = all_tags


# Tạo nhóm bằng cách tính khoảng cách cosine giữa các vector
def find_similar_tags(tag, tag_list, threshold=0.7):
    tag_vec = nlp(tag).vector
    similar_tags = [t for t in tag_list if nlp(t).vector.dot(tag_vec) > threshold]
    return similar_tags

# Ánh xạ tag vào nhóm
tag_groups = {}
for tag in tags:
    tag_groups[tag] = find_similar_tags(tag, tags)

print(tag_groups)

KeyboardInterrupt: 